In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import community
import pickle
import datetime

In [3]:
df = pd.read_pickle('samsung_co_inventor_data(2013-2014).pkl')
co_inventor_df = df.groupby('patent_id')['inventor_id'].apply(lambda x: list(x)).reset_index()

In [4]:
relationship = []

for i in range(len(co_inventor_df)):
    co_inventor_list = co_inventor_df.ix[i]['inventor_id']
    for idx, data in  enumerate(co_inventor_list):
        for j in co_inventor_list[idx:]:
            if data != j:
                relationship.append((data, j))

In [4]:
inventor_node = df.drop_duplicates(subset='inventor_id', keep='first').inventor_id.tolist()

In [5]:
G = nx.MultiGraph()
G.add_nodes_from(inventor_node)
G.add_edges_from(relationship)

In [6]:
i = 0

while(1):
    print(i,"번째",datetime.datetime.utcnow())
    i += 1
    
    remove_edge_list = []
    edge_btw = nx.edge_betweenness_centrality(G)
    
    if len(set(edge_btw.values())) == 1:
        break
    
    else:
        for k,v in edge_btw.items():
            if v == max(edge_btw.values()):
                remove_edge_list.append(k)
        G.remove_edges_from(remove_edge_list)

0 번째 2017-03-08 05:20:33.255071
1 번째 2017-03-08 05:20:35.963560
2 번째 2017-03-08 05:20:38.848025
3 번째 2017-03-08 05:20:41.616327
4 번째 2017-03-08 05:20:44.538586
5 번째 2017-03-08 05:20:47.290419
6 번째 2017-03-08 05:20:49.904757
7 번째 2017-03-08 05:20:52.454696
8 번째 2017-03-08 05:20:55.020748
9 번째 2017-03-08 05:20:57.593976
10 번째 2017-03-08 05:21:00.207593
11 번째 2017-03-08 05:21:02.775508
12 번째 2017-03-08 05:21:05.335419
13 번째 2017-03-08 05:21:07.914809
14 번째 2017-03-08 05:21:10.465428
15 번째 2017-03-08 05:21:13.020314
16 번째 2017-03-08 05:21:15.593988
17 번째 2017-03-08 05:21:18.222192
18 번째 2017-03-08 05:21:20.866435
19 번째 2017-03-08 05:21:23.551464
20 번째 2017-03-08 05:21:26.297327
21 번째 2017-03-08 05:21:28.889090
22 번째 2017-03-08 05:21:31.510888
23 번째 2017-03-08 05:21:34.096663
24 번째 2017-03-08 05:21:36.625260
25 번째 2017-03-08 05:21:39.267221
26 번째 2017-03-08 05:21:41.853642
27 번째 2017-03-08 05:21:44.463363
28 번째 2017-03-08 05:21:46.995576
29 번째 2017-03-08 05:21:49.612625
30 번째 2017-03-08 05:

In [7]:
dict_connected_graph = nx.to_dict_of_lists(G)

del_key = []
for k,v in dict_connected_graph.items():
    if v == []:
        del_key.append(k)
        
for k in del_key:
    del dict_connected_graph[k]

inventor = set(inventor_node) - set(del_key)

In [8]:
def connected_components(nodes):
    
    result = []
    nodes = set(nodes)
            
    while nodes:
        
        n = nodes.pop()
        group = {n}
        queue = [n]

        while queue:

            n = queue.pop(0)
            neighbors = set(dict_connected_graph[n])

            neighbors.difference_update(group)
            nodes.difference_update(neighbors)
            group.update(neighbors)
            queue.extend(neighbors)
            
        result.append(group)
    
    return result

In [9]:
all_group = []
number = 1
for components in connected_components(inventor):
    names = sorted(node for node in components)
    group = list(names)
    if len(group)>=3:
        all_group.append(group)
    number += 1

In [12]:
all_group

[['5381812-1',
  '5771350-1',
  '6735084-2',
  '8065778-3',
  '8224487-1',
  '8224487-10',
  '8224487-11',
  '8224487-6',
  '8224487-7',
  '8224487-8',
  '8224487-9'],
 ['4975944-1', '5473915-2', '6735588-1', '7834348-3', '8367444-5'],
 ['6024506-2', '7584401-3', '8191061-1', '8195054-3', '8570978-5'],
 ['7182819-1', '8487778-1', '8487778-3'],
 ['5825693-1',
  '6175675-2',
  '6944445-1',
  '7667266-3',
  '7781714-2',
  '7799643-2'],
 ['4983860-2', '5780325-1', '8884377-1', '8884377-4'],
 ['7019327-1', '7775782-2', '8609803-3', '8742410-4'],
 ['7544538-3', '7924492-2', '8816499-3', '8816499-4'],
 ['5193079-3',
  '6086443-3',
  '6106259-3',
  '6207488-1',
  '6501743-2',
  '6673718-4',
  '7222162-1',
  '7583095-1'],
 ['6262938-3', '6293830-8', '7299039-1', '7704788-2', '7909251-2'],
 ['5669513-4', '6215246-1', '6357076-1', '7245846-1'],
 ['5082694-2', '5888864-3', '6950929-2', '7053418-2', '7063997-1'],
 ['5543340-1', '7250788-2', '7450203-5'],
 ['7875340-4', '8142240-3', '8400082-1', '84

In [25]:
import pandas as pd
import pymysql

conn=pymysql.connect(host="192.9.65.20", user="jayoung", passwd="04tjvnvn23!", db="patent_DB", charset="utf8")

In [26]:
cursor = conn.cursor()
sql = "SELECT * FROM patent_DB.application_sample"
cursor.execute(sql)

field_names = [i[0] for i in cursor.description]
result = [value for value in cursor.fetchall()]

application_sample = pd.DataFrame(result, columns=field_names)

In [27]:
application_sample

,id,patent_id,type,number,country,date
0,08/778842,6069164,08,08778842,US,0000-00-00
1,02/002761,D345393,02,02002761,US,1992-12-21
2,02/007691,5164715,02,02007691,US,1990-04-10
3,02/010248,5177974,02,02010248,US,1988-06-23
4,02/020141,5379515,02,02020141,US,1994-02-16
5,02/027172,5264790,02,02027172,US,1991-07-01
6,02/030999,5300544,02,02030999,US,1993-03-12
7,02/033282,3955943,02,02033282,US,1948-06-16
8,02/039194,5363691,02,02039194,US,1993-04-14
9,02/039209,5380569,02,02039209,US,1993-04-08


In [47]:
cursor = conn.cursor()
sql = "SELECT * FROM patent_DB.application_update"
cursor.execute(sql)

field_names = [i[0] for i in cursor.description]
result = [value for value in cursor.fetchall()]

application_update = pd.DataFrame(result, columns=field_names)

In [51]:
application_update = application_update.drop('number',1)
application_update['number'] = application_update['number_updated'].apply(lambda x : str(x)[:-1])

In [52]:
application_update

,id,type,country,date,abstract,title,granted,num_claims,filename,number_updated,number
0,2001/20010000001,utility,US,2001-03-15,The invention provides a solvent mixture compr...,Solvent mixture for use in a vapor degreaser a...,1,16,pa010315.xml,09725796\r,09725796
1,2001/20010000002,utility,DE,2001-03-15,Process and a device for detecting a change in...,Process for detecting and correcting a fiber o...,1,33,pa010315.xml,09726341\r,09726341
2,2001/20010000003,utility,US,2001-03-15,A lift apparatus having a pair of support memb...,Adjustable scaffolding and lift carriage and s...,1,32,pa010315.xml,09728776\r,09728776
3,2001/20010000004,utility,US,2001-03-15,A snack dispenser consists of a series of tier...,Snack dispenser,1,46,pa010315.xml,09725772\r,09725772
4,2001/20010000005,utility,US,2001-03-15,A multicolor organic light emitting device emp...,Transparent contacts for organic devices,1,47,pa010315.xml,09726482\r,09726482
5,2001/20010000006,utility,FR,2001-03-15,Hydrocarbon feeds are subjected to hydrocracki...,Hydrocracking process with catalyst comprising...,1,15,pa010315.xml,09725131\r,09725131
6,2001/20010000007,utility,JP,2001-03-15,A method and apparatus are disclosed for treat...,Method and apparatus for treating organic wast...,1,15,pa010315.xml,09725780\r,09725780
7,2001/20010000008,utility,JP,2001-03-15,A method and apparatus are disclosed for treat...,Method and apparatus for treating organic wast...,1,15,pa010315.xml,09725785\r,09725785
8,2001/20010000009,utility,US,2001-03-15,A shunt controlled welding machine has a regul...,Planocentric gear for amperage indicator on we...,1,13,pa010315.xml,09725385\r,09725385
9,2001/20010000010,utility,US,2001-03-15,The present invention provides a code reader f...,Code reader,1,3,pa010315.xml,09725320\r,09725320


In [40]:
rawinventor_df = pd.read_csv('rawinventor.tsv',delimiter='\t',encoding='latin1')

In [41]:
rawinventor_df

,uuid,patent_id,inventor_id,rawlocation_id,name_first,name_last,sequence
0,000001d7-3c75-4ae7-aaa3-50aeacacd35e,8203015,5306849-1,c9d93742-bd11-4b6b-a9fd-49af25dbc6f8,Norbert,Lui,0
1,00000d0b-9842-4683-8094-bf9c1a857f3e,7883838,6599587-3,680c78d2-d43c-4d3d-b453-d1b1ae1fb15c,Young Hun,Byun,2
2,00000df8-4398-485f-81c2-7e8bb2f1c1c9,7708966,4127708-2,8a45c288-8bed-4275-b355-6875c2d19f3b,Ashok V.,Joshi,0
3,00001094-7d08-4466-a98f-495c7910a808,8230030,8230030-1,0723319e-02cf-4639-99a9-2f90d55def37,Joel George,Cummings,0
4,000010cc-c318-4295-8bae-89143aaf72bb,9327749,6840527-2,ca604acf-c4d8-46fc-a322-5647fca09d6e,Thomas K.,Schlegel,2
5,00001264-6e5d-44f3-a914-78fda18e7a5f,7872801,7872801-2,a79f6715-1e20-4008-baa6-1750e1517bea,Hayato,Takeuchi,1
6,000014c0-0251-42c0-925f-e0f816c68f72,7979462,7979462-3,9afc6571-8efa-4cd2-87a2-e004bc3433d4,Pedraum R.,Pardehpoosh,2
7,0000155b-478a-43c8-98d7-4973e3e71d32,8270425,7808898-2,18e2ef18-3a24-43b7-81f6-12ccd5476143,Ravindra,Guntur,1
8,00001e3a-2355-445d-894b-84533510612a,7700293,7229774-1,5f3da686-cf8f-409d-bf8d-794a731eb026,Arul M.,Chinnaiyan,0
9,00001e3e-59d6-49d2-b604-363ddab0c170,7837407,4273730-1,464a02ee-ef6e-4069-8e28-b9beeed93759,Thomas J.,Byrnes,5


In [53]:
merge_df = pd.merge(rawinventor_df, application_sample, on='patent_id')
merge_df = pd.merge(merge_df, application_update, on='number')

In [54]:
merge_df

,uuid,patent_id,inventor_id,rawlocation_id,name_first,name_last,sequence,id_x,type_x,number,...,id_y,type_y,country_y,date_y,abstract,title,granted,num_claims,filename,number_updated
0,000001d7-3c75-4ae7-aaa3-50aeacacd35e,8203015,5306849-1,c9d93742-bd11-4b6b-a9fd-49af25dbc6f8,Norbert,Lui,0,2010/12795168,12,12795168,...,2010/20100312002,utility,US,2010-12-09,The present invention relates to a process for...,Process For Preparing Fluoroalkyl Nitriles,1,9,ipa101209.xml,12795168\r
1,0354e965-f240-425e-8255-f1b4d1406e40,8203015,6541640-1,1711efb6-e17e-4194-984b-92d9a6e9439f,Sergii,Pazenok,3,2010/12795168,12,12795168,...,2010/20100312002,utility,US,2010-12-09,The present invention relates to a process for...,Process For Preparing Fluoroalkyl Nitriles,1,9,ipa101209.xml,12795168\r
2,0357b3d3-25df-416d-ab5b-1e32a801a549,8203015,7750186-2,b6dfc96e-1542-4542-9e0e-564a63e03abf,Jens-Dietmar,Heinrich,1,2010/12795168,12,12795168,...,2010/20100312002,utility,US,2010-12-09,The present invention relates to a process for...,Process For Preparing Fluoroalkyl Nitriles,1,9,ipa101209.xml,12795168\r
3,a60c9911-2ea8-4d8f-b990-f8594b4fb2f2,8203015,7977494-4,4685dbcf-e777-4ec7-94e2-338830b2c6cb,Thomas,Wollner,2,2010/12795168,12,12795168,...,2010/20100312002,utility,US,2010-12-09,The present invention relates to a process for...,Process For Preparing Fluoroalkyl Nitriles,1,9,ipa101209.xml,12795168\r
4,00000d0b-9842-4683-8094-bf9c1a857f3e,7883838,6599587-3,680c78d2-d43c-4d3d-b453-d1b1ae1fb15c,Young Hun,Byun,2,2003/10718809,10,10718809,...,2005/20050250051,utility,US,2005-11-10,An organometallic composition containing an or...,Organometallic composition for forming a metal...,1,11,ipa051110.xml,10718809\r
5,40c4baa5-87e9-43a3-8ba7-865aebe53c77,7883838,7005674-1,7ba2d3f8-73b4-4665-bdd7-928c83ab16cf,Sang Yoon,Lee,4,2003/10718809,10,10718809,...,2005/20050250051,utility,US,2005-11-10,An organometallic composition containing an or...,Organometallic composition for forming a metal...,1,11,ipa051110.xml,10718809\r
6,8d0a3058-39c2-41da-9904-bfc881ecef1c,7883838,6599587-4,e302978f-c286-4e51-85ea-576b4a199963,Euk Che,Hwang,3,2003/10718809,10,10718809,...,2005/20050250051,utility,US,2005-11-10,An organometallic composition containing an or...,Organometallic composition for forming a metal...,1,11,ipa051110.xml,10718809\r
7,921e48e9-65d6-4bef-9d30-6c1dbc241433,7883838,5338240-1,b9f22fa6-5f1f-49b6-bb5f-48f55842b69a,Jin Young,Kim,0,2003/10718809,10,10718809,...,2005/20050250051,utility,US,2005-11-10,An organometallic composition containing an or...,Organometallic composition for forming a metal...,1,11,ipa051110.xml,10718809\r
8,9423a5f8-e748-4570-8dd4-5e37540848fa,7883838,5441766-4,8ddd5001-e9e4-4738-9cad-1d18c2359f92,Soon Taik,Hwang,1,2003/10718809,10,10718809,...,2005/20050250051,utility,US,2005-11-10,An organometallic composition containing an or...,Organometallic composition for forming a metal...,1,11,ipa051110.xml,10718809\r
9,00000df8-4398-485f-81c2-7e8bb2f1c1c9,7708966,4127708-2,8a45c288-8bed-4275-b355-6875c2d19f3b,Ashok V.,Joshi,0,2008/12025423,12,12025423,...,2009/20090196810,utility,US,2009-08-06,A selective catalytic reduction (SCR) system i...,Systems and Methods for On-Site Selective Cata...,1,25,ipa090806.xml,12025423\r


In [ ]:
applli